<a href="https://colab.research.google.com/github/gmazzitelli/studenti/blob/master/CpuGpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
OSSYS=!uname -a | awk '{print $1}'
# "Darwin" o "Linux"
if OSSYS[0]=="Linux":
  cpu=!lscpu
  gpu=!nvidia-smi
if OSSYS[0]=="Darwin":
  cpu=!sysctl -a machdep.cpu
  gpu=!ioreg -l | grep num_cores
print(OSSYS)
print(cpu)
print(gpu)

['Linux']
['Architecture:        x86_64', 'CPU op-mode(s):      32-bit, 64-bit', 'Byte Order:          Little Endian', 'CPU(s):              2', 'On-line CPU(s) list: 0,1', 'Thread(s) per core:  2', 'Core(s) per socket:  1', 'Socket(s):           1', 'NUMA node(s):        1', 'Vendor ID:           GenuineIntel', 'CPU family:          6', 'Model:               79', 'Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz', 'Stepping:            0', 'CPU MHz:             2199.998', 'BogoMIPS:            4399.99', 'Hypervisor vendor:   KVM', 'Virtualization type: full', 'L1d cache:           32K', 'L1i cache:           32K', 'L2 cache:            256K', 'L3 cache:            56320K', 'NUMA node0 CPU(s):   0,1', 'Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe 

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.085042453000028
GPU (s):
0.0375277779999692
GPU speedup over CPU: 82x
